In [1]:
import numpy as np # linear algebra
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')


# from sklearn.linear_model import LogisticRegression ###---(분류 주의주의)
# -------------------------------------------------
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
# from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
# from xgboost import XGBClassifier
from xgboost import XGBRegressor
# from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor



from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error  #잔차(실값-예측값) 제곱 합의 평균
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
from scipy.stats import skew, kurtosis  #왜도(skew):좌우치우침    첨도(kurt):중앙값의뾰족함정도

from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV    # =StratifiedKFold 회귀에선 사용불가

sns.set()

# Data load

In [2]:
train = pd.read_csv("./dataset/train.csv")
test = pd.read_csv("./dataset/test.csv")
sub = pd.read_csv("./dataset/sample_submission.csv")

### train + test

In [3]:
train_test = pd.concat([train,  test], axis=0)

#  EDA

In [4]:
# train.columns

### 결측

In [34]:
print(train_test.isna().sum()[train_test.isna().sum()>0].sort_values(ascending=False).index.values.tolist())
# train_test.isna().sum()[train_test.isna().sum() >0].sort_values(ascending=False)

nan_df = pd.DataFrame({"col": train_test.columns, "sum":train_test.isna().sum() , "rate":np.round(train_test.isna().sum() / train_test.shape[0],2)*100})
nan_df[nan_df['sum']>0].sort_values(by='sum', ascending=False)

['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'SalePrice', 'FireplaceQu', 'LotFrontage', 'GarageCond', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageType', 'BsmtExposure', 'BsmtCond', 'BsmtQual', 'BsmtFinType2', 'BsmtFinType1', 'MasVnrType', 'MasVnrArea', 'MSZoning', 'Functional', 'BsmtHalfBath', 'BsmtFullBath', 'Utilities', 'KitchenQual', 'TotalBsmtSF', 'BsmtUnfSF', 'GarageCars', 'GarageArea', 'BsmtFinSF2', 'BsmtFinSF1', 'Exterior2nd', 'Exterior1st', 'SaleType', 'Electrical']


,col,sum,rate
PoolQC,PoolQC,2909,100.0
MiscFeature,MiscFeature,2814,96.0
Alley,Alley,2721,93.0
Fence,Fence,2348,80.0
SalePrice,SalePrice,1459,50.0
FireplaceQu,FireplaceQu,1420,49.0
LotFrontage,LotFrontage,486,17.0
GarageCond,GarageCond,159,5.0
GarageYrBlt,GarageYrBlt,159,5.0
GarageFinish,GarageFinish,159,5.0


In [23]:
obj_cols = train.select_dtypes(include='object').columns.values.tolist()
num_cols = train.select_dtypes(exclude='object').columns.values.tolist()

In [25]:
for col in obj_cols:
    print(f'{col} {train_test[col].nunique()}  {train_test[col].unique()[:10]}')

MSZoning 5  ['RL' 'RM' 'C (all)' 'FV' 'RH' nan]
Street 2  ['Pave' 'Grvl']
Alley 2  [nan 'Grvl' 'Pave']
LotShape 4  ['Reg' 'IR1' 'IR2' 'IR3']
LandContour 4  ['Lvl' 'Bnk' 'Low' 'HLS']
Utilities 2  ['AllPub' 'NoSeWa' nan]
LotConfig 5  ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope 3  ['Gtl' 'Mod' 'Sev']
Neighborhood 25  ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer']
Condition1 9  ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2 8  ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType 5  ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle 8  ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle 6  ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl 8  ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
Exterior1st 15  ['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkFace' 'WdShing' 'CemntBd'
 'Plywood' 'AsbShng' 'Stucco']
Exter

In [ ]:
train[train.duplicated(keep='last')].index

# dummy
* Creating dummy variables from categorical features
* Normalize skewed features using log_transformation


In [ ]:
# Converting non-numeric predictors stored as numbers into string

train_test['MSSubClass'] = train_test['MSSubClass'].apply(str)
train_test['YrSold'] = train_test['YrSold'].apply(str)
train_test['MoSold'] = train_test['MoSold'].apply(str)
train_test['OverallQual'] = train_test['OverallQual'].apply(str)
train_test['OverallCond'] = train_test['OverallCond'].apply(str)

#### Filling Categorical NaN

In [ ]:
train_test['Functional'] = train_test['Functional'].fillna('Typ')
train_test['Electrical'] = train_test['Electrical'].fillna("SBrkr")
train_test['KitchenQual'] = train_test['KitchenQual'].fillna("TA")
train_test['Exterior1st'] = train_test['Exterior1st'].fillna(train_test['Exterior1st'].mode()[0])
train_test['Exterior2nd'] = train_test['Exterior2nd'].fillna(train_test['Exterior2nd'].mode()[0])
train_test['SaleType'] = train_test['SaleType'].fillna(train_test['SaleType'].mode()[0])
train_test["PoolQC"] = train_test["PoolQC"].fillna("None")
train_test["Alley"] = train_test["Alley"].fillna("None")
train_test['FireplaceQu'] = train_test['FireplaceQu'].fillna("None")
train_test['Fence'] = train_test['Fence'].fillna("None")
train_test['MiscFeature'] = train_test['MiscFeature'].fillna("None")
for col in ('GarageArea', 'GarageCars'):
    train_test[col] = train_test[col].fillna(0)
        
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_test[col] = train_test[col].fillna('None')
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_test[col] = train_test[col].fillna('None')
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea','BsmtUnfSF', 'TotalBsmtSF'):
    train_test[col] = train_test[col].fillna(0)

train_test['LotFrontage'] = train_test['LotFrontage'].fillna(train['LotFrontage'].median())
    
    # Checking the features with NaN remained out

print(train_test[col].isna().sum()[train_test[col].isna().sum()>0])

In [ ]:
train_test_dummy = pd.get_dummies(train_test)
num_col_idx =  train_test_dummy.dtypes[train_test_dummy.dtypes != object].index

skew_df = train_test_dummy[num_col_idx].apply(lambda x: skew(x)).sort_values(ascending=False) 
for i in skew_df[skew_df > 0.5].index
    train_test_dummy[i] = np.log1p(train_test_dummy[i] ) 

## 파생변수

In [ ]:
train_test["SqFtPerRoom"] = train_test["GrLivArea"] / (train_test["TotRmsAbvGrd"] +
                                                       train_test["FullBath"] +
                                                       train_test["HalfBath"] +
                                                       train_test["KitchenAbvGr"])

train_test['Total_Home_Quality'] = train_test['OverallQual'] + train_test['OverallCond']

train_test['Total_Bathrooms'] = (train_test['FullBath'] + (0.5 * train_test['HalfBath']) +
                               train_test['BsmtFullBath'] + (0.5 * train_test['BsmtHalfBath']))

train_test["HighQualSF"] = train_test["1stFlrSF"] + train_test["2ndFlrSF"]
train_test['renovated']=train_test['YearRemodAdd']+train_test['YearBuilt']

In [ ]:
# useless = ['GarageYrBlt','YearRemodAdd'] 
# train_test = train_test.drop(useless, axis = 1)

### drop

## 평가

In [ ]:
def cv_rmse(model, X,y):
    kf =  KFold(n_splits=5, random_state=1414, shuffle=True)
    rmse = np.sqrt( -cross_val_score(model, X,y , scoring="neg_mean_squared_error",  cv=kf))
    return (rmse)

In [ ]:
cv_scores = []
cv_std = []
baseline_models = ['Linear_Reg.','Bayesian_Ridge_Reg.','LGBM_Reg.','SVR',
                   'Dec_Tree_Reg.','Random_Forest_Reg.', 'XGB_Reg.',
                   'Grad_Boost_Reg.','Cat_Boost_Reg.','Stacked_Reg.','Stacked_Reg2']

In [ ]:
lreg = LinearRegression()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
xgb = XGBRegressor()
gbr = GradientBoostingRegressor()
l_gbm = LGBMRegressor(objective='regression')

myparam = {'iterations': 6000,
          'learning_rate': 0.005,
          'depth': 4,
          'l2_leaf_reg': 1,
          'eval_metric':'RMSE',
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': 1414}
cat_f = CatBoostRegressor(**myparam)

catb = CatBoostRegressor()
stacking = StackingRegressor(regressors=(CatBoostRegressor(),XGBRegressor()),
                              meta_regressor = CatBoostRegressor(),
                              use_features_in_secondary = True)

models = [lreg = LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(), XGBRegressor(), GradientBoostingRegressor(), 
          LGBMRegressor(objective='regression'), CatBoostRegressor(),StackingRegressor()]
cv_score_list = []; cv_std_list = []
for model in models:
    score = cv_rmse(model, X, y_log)
    cv_score_list.append(score.mean())
    cv_std_list.append(score.std())

In [ ]:
cv_score_df = pd.DataFrame({'MODEL':models.__class_.__name,  'RMSE_MEAN':cv_scores , 'RMSE_STD'= cv_std})
# cv_score_df.plot(kind="bar")
cv_score_df

In [ ]:
cbr = CatBoostRegressor()
final_model = cbr.fit(X, y_log, plot=True)
feat_imp = final_model.get_feature_importance(prettified=True)
feat_imp.head()


In [ ]:
pred = cbr.predict(test)
sub['SalePrice'] = np.expm1(pred) 
sub.head()
sub.to_csv("cat_1.csv", index = False, header = True)
